

```
`# This is formatted as code`
```

# Detectron2

In [2]:
%cd "/content/drive/MyDrive/VBERT_ATTN_GRU/detectron2/"
!pip install -r requirements.txt
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# Install detectron2
!python setup.py build develop

/content/drive/MyDrive/VBERT_ATTN_GRU/detectron2
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-teerisiy
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-teerisiy
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-i7a0wq25
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-i7a0wq25
running build
running build_py
running build_ext
running develop
running egg_info
writing detectron2.egg-info/PKG-INFO
writing dependency_links to detectron2.egg-info/dependency_links.txt
writing requirements to detectron2.egg-info/requires.txt
writing top-level names to detectron2.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'detectron2.egg-info/SOURCES.txt'
running build_ext
copying build/lib.linux-x86_64-3.7/detectron2/_C.cpython-37m-x86_64-linux-gnu.so -> detectron2
Creating /usr/local/lib/python3.7/dist-packages/detectron2.egg-lin

In [2]:
import glob
import re
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import time
import cv2
import pickle
import os
import io
import torch
import tensorflow as tf

# import some common detectron2 utilities
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image

In [4]:
#Image and text
image_read = cv2.imread("test.png") #Path to image

In [5]:
NUM_OBJECTS = 36

# Load VG Classes
data_path = './demo/data/genome/1600-400-20'

vg_classes = []
with open(os.path.join(data_path, 'objects_vocab.txt')) as f:
    for object in f.readlines():
        vg_classes.append(object.split(',')[0].lower().strip())

MetadataCatalog.get("vg").thing_classes = vg_classes

#Config
cfg = get_cfg()
cfg.merge_from_file("./configs/VG-Detection/faster_rcnn_R_101_C4_caffe.yaml")
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 300
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
# VG Weight
cfg.MODEL.WEIGHTS = "http://nlp.cs.unc.edu/models/faster_rcnn_from_caffe.pkl"
predictor = DefaultPredictor(cfg)

def doit(raw_image):
    with torch.no_grad():
        final_array = []
        raw_height, raw_width = raw_image.shape[:2]
        #print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        #print("Transformed image size: ", image.shape[:2])
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Generate proposals with RPN
        proposals, _ = predictor.model.proposal_generator(images, features, None)
        proposal = proposals[0]
        proposal_boxes = [x.proposal_boxes for x in proposals]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        pred_class_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        outputs = FastRCNNOutputs(
            predictor.model.roi_heads.box2box_transform,
            pred_class_logits,
            pred_proposal_deltas,
            proposals,
            predictor.model.roi_heads.smooth_l1_beta,
        )
        probs = outputs.predict_probs()[0]
        boxes = outputs.predict_boxes()[0]
        
        for nms_thresh in np.arange(0.5, 1.0, 0.1):
            instances, ids = fast_rcnn_inference_single_image(
                boxes, probs, image.shape[1:], 
                score_thresh=0, nms_thresh=nms_thresh, topk_per_image=NUM_OBJECTS
            )
            if len(ids) == NUM_OBJECTS:
                break
                
        instances = detector_postprocess(instances, raw_height, raw_width)
        roi_features = feature_pooled[ids].detach()
        final_array.append(roi_features.cpu().numpy())
        final_array.append(instances.pred_boxes.tensor.cpu().numpy())
        final_array.append(raw_height)
        final_array.append(raw_width)
        return final_array

#Feature exrtraction
feature, bbox, img_w, img_h = doit(image_read)

#feature, bbox along with the report passed to the VBert model

WARNING [04/06 16:33:14 d2.config.compat]: Config '/content/drive/MyDrive/VBERT_ATTN_GRU/detectron2/configs/VG-Detection/faster_rcnn_R_101_C4_caffe.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/roi_heads.py):
	1. Change the stride of conv1 and shortcut in Res5.Block1 from 2 to 1.
	2. Modifying all conv2 with (padding: 1 --> 2) and (dilation: 1 --> 2).
	For more details, please check 'https://github.com/peteanderson80/bottom-up-attention/blob/master/models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'.

